In [21]:
import pandas as pd

In [8]:
text = open('data.txt').read()

In [22]:
import json
import spacy

In [28]:
with open('ner.json') as fp:
  training_data = json.load(fp)

In [53]:
train = training_data['annotations']

In [54]:
train

[['The retail arm of Reliance Industries Ltd Wednesday announced that Qatar’s sovereign wealth fund would invest Rs 8,278 crore in the company for a 0.99 per cent stake.The investment by Qatar Investment Authority (QIA) values Reliance Retail Ventures Ltd (RRVL) at a pre-money valuation of Rs 8.278 lakh crore, the company said in a release, making it among the country’s top four companies by equity value.We look forward to benefiting from QIA’s global experience and strong track record of value creation as we further develop RRVL into a world class institution, driving transformation of the Indian retail sector. The investment by QIA is a strong endorsement of a positive outlook towards the Indian economy and Reliance’s retail business model, strategy and execution capabilities,” said Isha Ambani, Director, RRVL.',
  {'entities': [[18, 41, 'COMPANY'],
    [42, 51, 'DAY'],
    [110, 124, 'MONEY'],
    [146, 159, 'PERCENT'],
    [224, 252, 'COMPANY'],
    [288, 307, 'MONEY'],
    [793, 8

In [40]:
nlp = spacy.load('en_core_web_sm')
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [41]:
ner = nlp.get_pipe('ner')

In [56]:
for _,annotation in train:
  for ent in annotation.get('entities'):
    ner.add_label(ent[2])

In [58]:
disable_pipes = [pipe for pipe in nlp.pipe_names if pipe!='ner']

In [59]:
disable_pipes

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer']

In [60]:
import random
from spacy.util import minibatch , compounding
from pathlib import Path

In [ ]:
nlp.update()

In [69]:
from spacy.training.example import Example

In [ ]:
with nlp.disable_pipes(*disable_pipes):
  optimizer = nlp.resume_training()

  for iteration in range(100):
    random.shuffle(train)
    losses = {}

    batches = minibatch(train, size=compounding(1.0, 4.0, 1.001))
    for batch in batches:
        examples = []
        for text, annotation in batch:
            example = Example.from_dict(nlp.make_doc(text), annotation)
            examples.append(example)

        # Update the model with the batch data.
        nlp.update(
            examples,
            drop=0.5,
            losses=losses,
            sgd=optimizer
        )
    # Print losses after processing each batch.
    print("Iteration", iteration + 1)
    print("Losses", losses)

In [75]:
for text,_ in train:
  doc = nlp(text)
  print('Entities',[(ent.text,ent.label_) for ent in doc.ents])

Entities [('Reliance Industries Ltd', 'COMPANY'), ('Wednesday', 'DAY'), ('Rs 8,278 crore', 'MONEY'), ('0.99 per cent', 'PERCENT'), ('Reliance Retail Ventures Ltd', 'COMPANY'), ('Rs 8.278 lakh crore', 'MONEY'), ('Isha Ambani', 'PERSON')]
